## Getting GTFS paths from files

This file is already generated, so no need to waste 20 minutes of your life

In [12]:
import dask.dataframe as dd
import pandas as pd
import re

# ------------------------------
# Define column types
# ------------------------------
stops_dtypes = {
    'stop_name': 'object',
    'parent_station': 'object',
    'stop_id': 'object',
    'stop_lat': 'float64',
    'stop_lon': 'float64',
    'location_type': 'float64',
    'platform_code': 'object'
}

routes_dtypes = {
    'route_long_name': 'object',
    'route_short_name': 'object',
    'agency_id': 'object',
    'route_type': 'float64',
    'route_id': 'object',
    'route_color': 'object',
    'route_text_color': 'object'
}

trips_dtypes = {
    'route_id': 'object',
    'service_id': 'object',
    'trip_id': 'object'
}

stop_times_dtypes = {
    'trip_id': 'object',
    'arrival_time': 'object',
    'departure_time': 'object',
    'stop_id': 'object',
    'stop_sequence': 'int64',
    'pickup_type': 'float64',
    'drop_off_type': 'float64'
}


In [ ]:
# ------------------------------
# Load CSVs with Dask
# ------------------------------
stops = dd.read_csv("stops.txt", dtype=stops_dtypes)
routes = dd.read_csv("routes.txt", dtype=routes_dtypes)
trips = dd.read_csv("trips.txt", dtype=trips_dtypes)
stop_times = dd.read_csv("stop_times.txt", dtype=stop_times_dtypes)

In [ ]:
# Merge stop_times with trips to get route_id for each stop
stop_times_with_route = stop_times.merge(trips[['trip_id', 'route_id']], on='trip_id', how='left')

# Merge with stops to get stop_name
stop_times_with_route = stop_times_with_route.merge(stops[['stop_id', 'stop_name']], on='stop_id', how='left')
stop_times_with_route = stop_times_with_route.merge(routes[['route_id', 'route_short_name']], on='route_id', how='left')

# Group by route_id and aggregate stop_name into a list (ordered by stop_sequence)
route_stops = ( stop_times_with_route.sort_values(['route_id', 'trip_id', 'stop_sequence']).groupby(['route_short_name','route_id','trip_id'])['stop_name'].apply(list).reset_index() )
route_stops = route_stops.compute()

# Optional: remove duplicates within each route
route_stops['stop_name'] = route_stops['stop_name'].apply(lambda x: list(dict.fromkeys(x)))


C:\Users\neilinn\AppData\Local\Temp\ipykernel_28428\1989338775.py:9: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result

  route_stops = ( stop_times_with_route.sort_values(['route_id', 'trip_id', 'stop_sequence']).groupby(['route_short_name','route_id','trip_id'])['stop_name'].apply(list).reset_index() )


In [ ]:
def clean_station_name(station: str) -> str:
    """Clean one station name: remove hyphens, brackets, and trim spaces."""
    if pd.isna(station):
        return ""
    station = station.replace('-', ' ')
    station = station.replace('(', ' ')
    station = station.replace(')', ' ')
    station = station.replace('Hauptbahnhof', 'Hbf')
    station = re.sub(r'\s+', ' ', station)
    return station.strip()

# If stop_name column is a list of stations
#route_stops['stop_name_fixed'] = route_stops['stop_name'].apply(lambda stops: '|'.join(clean_station_name(s) for s in stops))
route_stops['stop_name_fixed'] = route_stops['stop_name'].apply(lambda stops: [clean_station_name(s) for s in stops])

In [ ]:
gtfs_file = route_stops[['stop_name_fixed', 'route_short_name']]
gtfs_file['stop_name_fixed'] = gtfs_file['stop_name_fixed'].apply(lambda stops: '|'.join(s for s in stops))
gtfs_file.to_csv("gtfs_paths.csv", index=False)

## Generating all possible paths

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

data = pd.read_csv('DBtrainrides.csv', encoding='UTF-8')

In [ ]:
def clean_station_name(station: str) -> str:
    """Clean one station name: remove hyphens, brackets, and trim spaces."""
    station = station.replace('-', ' ')       # replace hyphens with spaces
    station = station.replace('(', ' ')    # replace brackets with spaces
    station = station.replace(')', ' ')    # replace brackets with spaces
    station = station.replace('Hauptbahnhof', 'Hbf')
    station = re.sub(r'\s+', ' ', station)
    return station.strip()

def clean_path(path: str) -> str:
    """Clean a full path, applying cleaning per station."""
    if not isinstance(path, str) or not path.strip():
        return ''
    stations = path.split('|')
    cleaned = [clean_station_name(s) for s in stations]
    return '|'.join(cleaned)

# Apply cleaning
data['path'] = data['path'].apply(clean_path)

In [ ]:
data['station'] = data['station'].apply(clean_path)

In [ ]:
all_full_routes = []

##each station in path do the clean, right now whole path is being cleaned
line_numbers = data.loc[data['line'].astype(str).str.match(r'^\d+[a-zA-Z]?$'), 'line'].unique().tolist()

for number in line_numbers:
    subset = data[data['line'].astype(str) == str(number)]

    #subset = data[data['line'].astype(str).match(r'^\d+[a-zA-Z]?$')]
    # Filter paths starting with Rostock Hbf
    #subset = subset[subset['path'].astype(str).str.strip().str.lower().str.startswith('rostock hbf')]

    # Create full path by appending the next station
    subset['full_path'] = subset['path'].fillna('').astype(str).str.strip()
    subset['full_path'] = subset.apply(
        lambda row: row['full_path'] + '|' + row['station'] if row['full_path'] else row['station'],
        axis=1
    )

    # Drop duplicates to speed up the next step
    subset = subset.drop_duplicates(subset=['full_path']).reset_index(drop=True)

    # Sort the rows by path length to potentially make it faster
    subset['path_length'] = subset['full_path'].str.count(r'\|') + 1
    subset = subset.sort_values('path_length').reset_index(drop=True)

    # Identify full routes: a path is full if no other path starts with it
    all_paths = subset['full_path'].tolist()
    full_routes = [p for i, p in enumerate(all_paths) 
                if not any(other != p and other.startswith(p + '|') for other in all_paths)]

    # Convert to DataFrame
    df = pd.DataFrame(full_routes, columns=['full_path'])
    df['line'] = number
    all_full_routes.append(df)

full_routes_df = pd.concat(all_full_routes, ignore_index=True)
# Save the full_routes_df to a CSV file
# Add an empty column called 'fixed_line' to the DataFrame
full_routes_df['fixed_line'] = ''
full_routes_df.to_csv('full_routes.csv', index=False)

## Mapping lines with corresponding path

In [ ]:
full_routes = pd.read_csv("full_routes.csv")

# Clean and split 'full_path' into a list at '|'
full_routes['full_path'] = (
    full_routes['full_path']
    .str.replace('Hauptbahnhof', 'Hbf')
    .str.strip()
    .apply(lambda x: [s.strip() for s in x.split('|') if s.strip()])
)

route_lookup = {}

for _, row in route_stops.iterrows():
    stops = tuple(row['stop_name_fixed'])
    reversed_stops = tuple(row['stop_name_fixed'][::-1])
    route_name = row['route_short_name']

    # Add both directions to the lookup
    route_lookup[stops] = route_name
    route_lookup[reversed_stops] = route_name

# Match each full_path_clean to its route_short_name if found
def find_route_name(path_list):
    """Return route_short_name if list (or its reverse) matches any route."""
    return route_lookup.get(tuple(path_list), None)

full_routes['fixed_line'] = full_routes['full_path'].apply(find_route_name)
full_routes['full_path'] = full_routes['full_path'].apply(lambda stops: '|'.join(s for s in stops))
full_routes.to_csv("full_routes_mapped1.csv", index=False)

In [24]:
empty_count = (full_routes['fixed_line'].isna() | (full_routes['fixed_line'] == '')).sum()
print(empty_count)


6347


In [101]:
pd.set_option('display.max_colwidth', None)
file_paths = pd.read_csv("gtfs_paths.csv")

filtered_rows = file_paths[
    file_paths['stop_name_fixed'].str.contains('Stuttgart Schwabstraße', na=False) & 
    file_paths['route_short_name'].str.contains('1', na=False)
]['route_short_name'].unique()

print(filtered_rows)

[]


In [145]:
pd.set_option('display.max_colwidth', None)
file_paths = pd.read_csv("gtfs_paths.csv")

filtered_rows = file_paths[
    file_paths['stop_name_fixed'].str.contains('München Rosenheimer Platz|München Ost|München Leuchtenbergring', na=False, regex=False) & 
    file_paths['route_short_name'].str.contains('4', na=False)
]['route_short_name'].unique()

print(filtered_rows)

['439' 'S4']


In [110]:
pd.set_option('display.max_colwidth', None)
gtfs_paths = pd.read_csv("gtfs_paths.csv")
line_mappings = pd.read_csv("correct_line_mapping.csv")
extra = pd.read_csv("correct_line_mapping.csv")
pattern = re.compile(r'^\d+[a-zA-Z]?$')

#line_mappings = line_mappings[line_mappings['line'].isin(['1', '20', '18'])]
#extra = extra[extra['line'].isin(['1', '20', '18'])]
empty_count = (line_mappings['fixed_line'].isna() | (line_mappings['fixed_line'] == '')).sum()
print(empty_count)

for i, row in line_mappings.iterrows():
    if pd.notna(row['fixed_line']) and row['fixed_line'] != '':
        continue

    filtered_rows = extra[
        extra['full_path'].str.contains(row['full_path'], na=False) & 
        extra['fixed_line'].str.match(rf'^[A-Za-z]*{row['line']}$')
    ]['fixed_line'].unique()

    filtered = [x for x in filtered_rows if not pattern.match(x)]

    if len(filtered) == 1:
        line_mappings.loc[i,'fixed_line'] = filtered[0]

line_mappings.to_csv("full_routes_mapped1.csv", index=False)

empty_count = (line_mappings['fixed_line'].isna() | (line_mappings['fixed_line'] == '')).sum()
print(empty_count)


94
94


In [72]:
pd.set_option('display.max_colwidth', None)
gtfs_paths = pd.read_csv("gtfs_paths.csv")
line_mappings = pd.read_csv("correct_line_mapping.csv")
extra = pd.read_csv("correct_line_mapping.csv")
pattern = re.compile(r'^\d+[a-zA-Z]?$')

#line_mappings = line_mappings[line_mappings['line'].isin(['1', '20', '18'])]
#extra = extra[extra['line'].isin(['1', '20', '18'])]
empty_count = (line_mappings['fixed_line'].isna() | (line_mappings['fixed_line'] == '')).sum()
print(empty_count)

for i, row in line_mappings.iterrows():
    if pd.notna(row['fixed_line']) and row['fixed_line'] != '':
        continue

    filtered_rows = gtfs_paths[
        gtfs_paths['stop_name_fixed'].str.contains(row['full_path'], na=False) & 
        gtfs_paths['route_short_name'].str.match(rf'^[A-Za-z]*{row['line']}$')
    ]['route_short_name'].unique()

    filtered = [x for x in filtered_rows if not pattern.match(x)]

    if len(filtered) == 1:
        line_mappings.loc[i,'fixed_line'] = filtered[0]

line_mappings.to_csv("full_routes_mapped1.csv", index=False)

empty_count = (line_mappings['fixed_line'].isna() | (line_mappings['fixed_line'] == '')).sum()
print(empty_count)

184
158


In [146]:

line_mappings = pd.read_csv("correct_line_mapping.csv")
line_mappings['is_empty'] = line_mappings['fixed_line'].isna() | (line_mappings['fixed_line'] == '')
empty_counts_per_line = line_mappings.groupby('line')['is_empty'].sum()
empty_counts_per_line = empty_counts_per_line[empty_counts_per_line > 0]
print(empty_counts_per_line)

Series([], Name: is_empty, dtype: int64)


In [147]:
empty_count = (line_mappings['fixed_line'].isna() | (line_mappings['fixed_line'] == '')).sum()
print(empty_count)


0
